In [1]:
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import time
import pymysql
import datetime
from selenium.webdriver.common.by import By
import warnings
import os
import random
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from pyautogui import alert as printw
from pyautogui import confirm
warnings.filterwarnings('ignore')

In [2]:
def find_by_text(group,text):
    for i in group:
        if i.text==text:
            return i

# 参数

In [3]:
accout ='data@1140822532457527.onaliyun.com'
password = 'Yy@321456'
query_id = 7913
bynd = 2023

# 下面是湖南省问卷要用的，不是湖南省问卷把下面注释掉
#query_id = '郴州职业技术学院'#已与老师确认-1109
#query_id = '湖南科技学院'#已与老师确认-1109
# query_id = '湖南女子学院'#已与老师确认-1109
#query_id = '湖南工学院'#已与老师确认-1109
#query_id = '湖南体育职业学院'#已与老师确认-1109
#query_id = '湖南科技大学'#已与老师确认-1109
#query_id = '湖南交通工程学院'#已与老师确认-1109
#query_id = '湖南文理学院'#【绝对不能删的邮箱】已与老师确认-1109
#query_id = '湖南九嶷职业技术学院'#【绝对不能删的邮箱】已与老师确认-1109
#query_id = '邵阳学院'#【绝对不能删的邮箱】已与老师确认-1109
#query_id = '长沙理工大学城南学院'#【绝对不能删的邮箱】已与老师确认-1109
#query_id = '湖南大众传媒职业技术学院'#已与老师确认-1109

#query_id = '湖南应用技术学院'
#query_id = '湖南信息职业技术学院'
#query_id = '湖南食品药品职业学院'
#query_id = '长沙卫生职业学院'
#query_id = '湖南中医药大学'
#query_id = '湖南交通职业技术学院'
#query_id = '长沙商贸旅游职业技术学院'
#query_id = '湖南警察学院'
#query_id = '湖南化工职业技术学院'

#query_id = '湖南师范大学'#【朱黎的不要动】
#query_id = '湖南工商大学'
#query_id = '湖南工业大学科技学院'
#query_id = '湖南幼儿师范高等专科学校'
#query_id = '湘中幼儿师范高等专科学校'
#query_id = '湖南汽车工程职业学院'
#query_id = '保险职业学院'
#query_id = '湘西民族职业技术学院'
#query_id = '湖南财政经济学院'
#query_id = '湖南财经工业职业技术学院'
#query_id = '吉首大学'
#query_id = '怀化职业技术学院'
#query_id = '湖南医药学院'

#query_id = '湖南邮电职业技术学院'#待确认
#query_id = '湖南铁道职业技术学院'
#query_id = '湘南学院'
#query_id = '娄底职业技术学院'
#query_id = '湖南人文科技学院'
#query_id = '湖南高速铁路职业技术学院'
#query_id = '永州师范高等专科学校'
#hunan_url = 'http://wshy.hunbys.net/survey/getCode?surveyFcode=f406a549875345dc89f351099cfd65ae'

# 根据id查相关信息

In [42]:
can_send = False
hunan_diaoyan = False
if isinstance(query_id,str):
    hunan_diaoyan = True
conn = pymysql.connect(host = 'am-wz9el267w54i2r7ip131930o.ads.aliyuncs.com' # 连接名称，默认127.0.0.1
    ,user = 'yjy_user' # 用户名
    ,passwd='Yjy123456' # 密码
    ,port= 3306 # 端口，默认为3306
    ,db='yun-campusjob'#数据库名称
    ,charset='utf8' # 字符编码
    )
if not hunan_diaoyan:
    # 云研问卷查一下id的信息
    sql = f'''select b.school_name,a.name,a.type,a.open_url,a.qr_code_url from sch_q_define a
    left join sch_school b on a.school_id = b.school_id
    where q_define_id = '{query_id}'
    '''
    data = pd.read_sql(con=conn,sql=sql)

curr_time = datetime.datetime.now()
time_str = curr_time.strftime("%Y年%m月")
time_str1 = curr_time.strftime("%m月%d日%H:%M:%S")

str1 = is_dw and '单位' or '学生'
row = hunan_diaoyan and [query_id,f'关于开展{bynd}届毕业生就业质量调研通知','0',hunan_url,'none'] or data.values[0]
temp1 = re.sub('\D','',row[1])
if int(bynd) != int(temp1):
    printw(f'届别对不上')
    raise
    
yunyan_flag = row[0]=='云研数据研究院'
row[0] = yunyan_flag and re.findall(f'(.*){bynd}',row[1])[0] or row[0]

if row[2] not in ['0','1']:
    row[2] = confirm(text='请手动选择问卷类型',title="无法根据id判断问卷类型", buttons=['学生问卷', '单位问卷']) =='学生问卷' and '0' or '1'
    
is_dw = row[2]=='1'

if hunan_diaoyan:
    # 湖南省问卷会单独跑一份学生邮箱出来
    sql = f'''select distinct a.school_id,email from sch_school_student a
join sch_school b on a.school_id =b.school_id
where is_yun=1 and is_online=1 and is_delete=0 and school_name = '{query_id}' and graduate_year = {bynd}
union
select school_id,'zzzzzz'||school_id||'@bibibi.net' from sch_school
where school_name = '{query_id}'
'''
    data_email = pd.read_sql(con=conn,sql=sql)
else:
    # 如果是云研的单位问卷要跑一份单位邮箱
    if is_dw and not yunyan_flag:
        sql = f'''
    select distinct b.MAIL email from c_sch_income a
    join c_company_c b on a.company_id = b.company_id
    join sch_school c on c.school_id = a.school_id
    where  a.is_delete=0 and a.is_blacklist=0 and a.state='已通过' and c.is_yun=1 and c.is_online=1 and school_name = '{row[0]}'
    union
    select 'zzzzzzdw{query_id}@bibibi.net' from sch_school
    '''
        data_email = pd.read_sql(con=conn,sql=sql)

# 邮箱文字部分

In [5]:
# 文字部分
temp = f'''<p style="text-indent:32px;">
    为了解你们就业的真实情况，母校联合第三方机构正在开展{temp1}届毕业生就业质量调研。

问卷会通过微信、邮件等形式发送到您的手中，请您认真作答，您答题的真实性对于帮助母校进一步改进学生就业创业指导服务工作以及完善人才培养方案具有至关重要的意义。

本次信息收集仅用于我校开展相关分析及研究，数据会严格保密。本次调研大约占用您3-10分钟的时间。

</p>'''
if not is_dw:
    text2 = temp
    text3 = '亲爱的校友'
else:
    text2 = f'''<p style="text-align: left; text-indent: 32px;">
    真诚地感谢您一直以来对我校毕业生就业工作的关注和支持！为更好地服务社会，培养适合企事业单位发展的人才，我校特开展本次调研。

本次调研主要涉及单位的人才需求、对毕业生的能力期望以及对我校教育教学的评价，大约会花费您五分钟的时间，数据将严格保密仅供统计分析之用。
</p>'''
    text3 = '尊敬的用人单位'

mystr = hunan_diaoyan and f'''<div style="width: 600px;padding: 10px 20px;line-height: 1.6;margin: 0 auto;background: #fbfbfb;">
    <p style="text-align:center;font-size: 24px;font-weight: bold;">
        {row[1]}
    </p>
    <p style="font-weight: bold;">
        亲爱的同学：
    </p>
    <p style="text-indent:32px;font-weight: bold;">
        您好！
    </p>
    <p style="text-indent:32px;">
        首先，请接受来自母校的诚挚问候和真切祝福！

时光匆匆，白驹过隙。您在母校度过了宝贵的青春时光，如今已登上人生的新舞台，母校将一如既往地关注您的成长。
为了解您目前的就业现状以及您对母校人才培养、管理服务的评价及反馈意见，湖南省就业指导中心下发了&ldquo;{bynd}届毕业生就业质量调研问卷&rdquo;，在此诚挚邀请您参与调查！您的回答对母校非常重要，答案无对错之分。
调研结果仅用于学校的分析研究，请放心填答。谢谢您对母校就业工作和人才培养的支持！ 母校期待着您常回来看看，也祝福您在人生的道路上一帆风顺，前程似锦！
    </p>
    <p style="text-indent:32px;">
        填写方式：
    </p>
    <p style="text-indent:32px;">
        (1)点击问卷地址直接访问填写（如无法打开，请将链接复制到微信打开即可），问卷地址：{hunan_url}
    </p>
    <p style="text-indent:32px;">
        (2)进入公众号填写：查找关注&ldquo;湘促就业&rdquo;微信公众号，进入公众号的&ldquo;服务中心&rdquo;菜单的&ldquo;毕业生服务&rdquo;栏目，选择&ldquo;{bynd}届毕业生就业质量调查&rdquo;进入填写问卷。
    </p>
    <p style="text-align:right;font-weight: bold;">
        {time_str}
    </p>
    <p></p>
    <p></p>
</div>''' or f'''<div style="width: 600px;padding: 10px 20px;line-height: 1.6;margin: 0 auto;background: #fbfbfb;">
<p style="text-align:center;font-size: 24px;font-weight: bold;">
    {row[1]}
</p>
<p style="font-weight: bold;">
    {text3}：
</p>
<p style="text-indent:32px;font-weight: bold;">
    您好！
</p>
{text2}
<p style="text-indent:32px;">
    真诚感谢您的理解与配合！
</p>
<p style="text-indent:32px;">
    问卷填写地址为：<a href="{row[3]}">点击打开问卷填写</a>。
</p>
<p style="text-indent:32px;">
    或扫描下方二维码进行填写。再次感谢您的参与！
</p>
<p style="text-align:center;">
    <img src="https://o.bysjy.com.cn/{row[4]}"/>
&nbsp; &nbsp;
</p>
<p style="text-align:right;font-weight: bold;">
    {time_str}
</p>
</div>'''

model_name = f'{row[0]}{time_str1}【{str1}】'[:30]
printw(f'请确定发件人名称——{row[0]}')

'OK'

# 打开浏览器

In [6]:
# 目前有发现需要向右滑动的时候会失败，可以关掉浏览器再试几次看看

options = webdriver.ChromeOptions()
# 添加无界面参数
# options.add_argument('--headless')
driver=webdriver.Chrome()
driver.implicitly_wait(10)
driver.get(r'https://signin.aliyun.com/login.htm?callback=https%3A%2F%2Fdm.console.aliyun.com%2F#/main')
# driver.maximize_window()
driver.find_element(By.NAME,"username").send_keys(accout)
driver.find_element(By.XPATH,'//*[@id="--aliyun-xconsole-app"]/div[1]/div[2]/div/div/div[1]/div[2]/div/div/div/div/div/form/div[5]/button').click()
driver.find_element(By.NAME,"password").send_keys(password)
driver.find_element(By.XPATH,'//*[@id="--aliyun-xconsole-app"]/div[1]/div[2]/div/div/div[1]/div[2]/div/div/div/div/div/form/div[5]/button').click()

# 新建模板

In [7]:
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[1]/div/div[1]/div[3]/ul/li[2]/ul/li[4]/div/a/div[2]').click()

# 查看是否超过40封模板，超过了自动删掉后9个模板
str2 = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[2]/div/div/div[2]/table/tfoot/tr/td/div[2]/div/div/span[1]').text
if len(re.findall('100',str2))==1:
    find_by_text(driver.find_element(By.CSS_SELECTOR,'[class="pull-right ng-isolate-scope"]').find_elements(By.CSS_SELECTOR,'[class="ng-binding"]'),'»').click()
    time.sleep(1)
    for i in range(9):
        time.sleep(1)
        driver.find_elements(By.CSS_SELECTOR,'[class="btn btn-link btn-xs ng-scope ng-isolate-scope"]')[-1].click()
        driver.find_element(By.CSS_SELECTOR,'[class="btn ng-scope ng-binding btn-primary"]').click()

# 新建模板
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[1]/div[2]/button[1]').click()
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[2]/div[2]/form/div/div[1]/input').send_keys(model_name)
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[2]/div[3]/form/div[1]/div[1]/input').send_keys(f'{row[1]}')
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[2]/div[3]/form/div[2]/div[1]/input').send_keys(f'{row[0]}')
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[2]/div[3]/form/div[4]/div[1]/div/div[1]/div[1]/div[1]/div[1]').click()

driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[2]/div[3]/form/div[4]/div[1]/div/div[2]/textarea[2]').clear()
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[2]/div[3]/form/div[4]/div[1]/div/div[2]/textarea[2]').send_keys(mystr)
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[3]/button[1]').click()
#提交审核
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[2]/div/div/div[2]/table/tbody/tr[1]/td[5]/div/a[2]/span').click()
driver.find_element(By.CSS_SELECTOR,'[class="btn ng-scope ng-binding btn-primary"]').click()

# 发送测试邮件

In [37]:
#点发送邮件新建任务
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[1]/div/div[1]/div[3]/ul/li[3]/ul/li[2]/div/a/div[2]').click()
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[1]/div[2]/button[1]').click()
time.sleep(1)
# 这里的下拉框都不支持点击，只能用action点
text_list = driver.find_elements(By.CLASS_NAME,'col-sm-5')
action = ActionChains(driver)
time.sleep(1)
#模板名称
action.click(driver.find_element(By.CLASS_NAME,'select2-chosen')).perform()
driver.find_element(By.XPATH, f"//span[@title='{model_name}']").click()
# 其他下拉框
action.click(text_list[2]).perform()
action.key_down(Keys.DOWN).key_down(Keys.ENTER).perform()
element1 = text_list[3].find_element(By.NAME,'addressType')
action.click(element1).perform()
action.key_down(Keys.DOWN).key_down(Keys.ENTER).perform()
action.click(text_list[4]).perform()
action.key_down(Keys.DOWN).key_down(Keys.ENTER).perform()
action.click(text_list[0]).perform()#收件人列表大部分情况下都是选测试
for i in range(10):#按试下down肯定没问题
    action.key_down(Keys.DOWN).perform()
action.key_down(Keys.DOWN).key_down(Keys.ENTER).perform()

# 新建收件人

In [44]:
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[1]/div/div[1]/div[3]/ul/li[3]/ul/li[1]/div/a/div[2]').click()
#看一下有没有满10个
time.sleep(1)#等全部加载出来
if len(driver.find_elements(By.CSS_SELECTOR,'[class="title-column"]')) >20:
    #删倒数第二个
    driver.find_elements(By.CSS_SELECTOR,'[class="btn btn-link btn-xs ng-scope ng-isolate-scope"]')[-3].click()
    driver.find_element(By.CSS_SELECTOR,'[class="btn ng-scope ng-binding btn-primary"]').click()

if hunan_diaoyan or (is_dw and not yunyan_flag):
    txt_name = hunan_diaoyan and f'{bynd}{query_id}学生邮箱.txt' or f'{bynd}{row[0]}单位邮箱.txt'
    temp = data_email.query('email.isnull() | email==""').shape[0]
    if txt_name not in os.listdir('.'):
        if temp!=0:
            printw(f'(总条数有{data_email.shape[0]}条，缺少{temp}条邮箱)')
        e_group=[]
        for email in data_email['email'].tolist():
            try:
                if not re.findall('@',email):continue
                if 'qq' in email or 'QQ' in email:
                    if re.findall('\d+@',email):
                        e_group.append(re.findall('\d+@',email)[0]+'qq.com')
                else:
                    e_group.append(email)
            except:
                pass
        pd.DataFrame(list(set(e_group))).to_csv(txt_name,sep='\n',index=False, header = False)
    else:
        print('上次跑过了这次就不跑了')
        
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[1]/div[2]/button[1]/span').click()
driver.find_element(By.CSS_SELECTOR,'[class="form-control  ng-pristine ng-invalid ng-invalid-required ng-valid-maxlength ng-valid-minlength"]').send_keys(model_name)# 列表名
random_num = random.randint(1,100000)
driver.find_element(By.CSS_SELECTOR,'[class="form-control  ng-pristine ng-invalid ng-invalid-required ng-valid-maxlength ng-valid-minlength ng-valid-pattern"]').send_keys(f'cs{random_num}@tsmail.bibibi.net')

driver.find_element(By.CSS_SELECTOR,'[class="form-control  ng-pristine ng-invalid ng-invalid-required ng-valid-maxlength ng-valid-minlength"]').send_keys(str1)

driver.find_element(By.CSS_SELECTOR,'[class="btn btn-primary ng-scope ng-binding"]').click()
time.sleep(2)
# 上传收件人
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[4]/div/div/div[2]/table/tbody/tr[1]/td[6]/div/a[1]/div').click()

printw(f'注意找这个学校！\n*****{row[0]}*****')

上次跑过了这次就不跑了


'OK'

In [45]:
# 测试收件人
for i in driver.find_element(By.CSS_SELECTOR,'[class="table table-hover"]').find_elements(By.CSS_SELECTOR,'[class="ng-scope"]'):
    if model_name in i.text:
        i.find_element(By.CSS_SELECTOR,'[class="btn btn-link btn-xs ng-scope ng-isolate-scope"]').click()
        break

driver.find_element(By.CSS_SELECTOR,'[class="form-inline"]').find_element(By.CSS_SELECTOR,'input').send_keys('zzzzzz')

find_by_text(driver.find_elements(By.CSS_SELECTOR,'[class="btn btn-primary ng-binding"]'),'查询').click()
time.sleep(2)
find_by_text(driver.find_elements(By.CSS_SELECTOR,'[class="btn btn-primary ng-binding"]'),'查询').click()
time.sleep(2)
id_email = driver.find_element(By.CSS_SELECTOR,'[class="modal-body"]').find_element(By.CSS_SELECTOR,'[class="table table-hover"]').find_element(By.CSS_SELECTOR,'[class="ng-scope"]').find_element(By.CSS_SELECTOR,'[class="title-column"]').text

if not is_dw:
    if 'dw' in id_email:
        printw('邮箱是单位邮箱！')
        raise
    if not hunan_diaoyan and int(re.sub('\D','',id_email))!=query_id:
        printw('邮箱不是该学校的学生的邮箱！')
        raise

    if hunan_diaoyan and int(re.sub('\D','',id_email))!=data_email['school_id'][0]:
        printw('邮箱不是该学校的学生的邮箱！')
        raise
if is_dw:
    if 'dw' not in id_email:
        printw('邮箱是学生邮箱！')
        raise
    if int(re.sub('\D','',id_email))!=query_id:
        printw('邮箱不是该学校的单位的邮箱！')
        raise
        
can_send = True
driver.find_element(By.CSS_SELECTOR,'[class="modal-footer"]').find_element(By.CSS_SELECTOR,'[class="btn btn-default ng-binding"]').click()

# 发送正式邮件

In [46]:
if not can_send:
    printw('测试未通过，不许发')
    raise
#点发送邮件新建任务
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[1]/div/div[1]/div[3]/ul/li[3]/ul/li[2]/div/a/div[2]').click()
driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div[3]/div[2]/div[1]/div[2]/div/div/div[1]/div[2]/button[1]').click()
time.sleep(1)
# 这里的下拉框都不支持点击，只能用action点
text_list = driver.find_elements(By.CLASS_NAME,'col-sm-5')
action = ActionChains(driver)
time.sleep(1)
#模板名称
action.click(driver.find_element(By.CLASS_NAME,'select2-chosen')).perform()
driver.find_element(By.XPATH, f"//span[@title='{model_name}']").click()
# 其他下拉框
action.click(text_list[2]).perform()
action.key_down(Keys.DOWN).key_down(Keys.ENTER).perform()
element1 = text_list[3].find_element(By.NAME,'addressType')
action.click(element1).perform()
action.key_down(Keys.DOWN).key_down(Keys.ENTER).perform()
action.click(text_list[4]).perform()
action.key_down(Keys.DOWN).key_down(Keys.ENTER).perform()
action.click(text_list[0]).perform()#收件人列表大部分情况下都是选测试
a = driver.find_element(By.XPATH, f"//option[text()='{model_name}']").get_attribute('value')
for i in range(int(a)):
    action.key_down(Keys.DOWN).perform()
    time.sleep(0.1)
action.key_down(Keys.DOWN).key_down(Keys.ENTER).perform()